In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


#1 Thư viện

In [ ]:
!pip install lightgbm==3.3.5

In [ ]:
pip install catboost

In [ ]:
pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures, StandardScaler, Normalizer, LabelEncoder, OneHotEncoder
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb

#2 Khai báo biến hệ thống

In [ ]:
DRIVE_PATH = '../gdrive/MyDrive/[IE224]_Final_Project/'
DATA_PATH = 'Data/final_data.csv'

In [ ]:
cd '../gdrive/MyDrive/[IE224]_Final_Project/'

[Errno 2] No such file or directory: '../gdrive/MyDrive/[IE224]_Final_Project/'
/gdrive/.shortcut-targets-by-id/1z39zLU-VuS7BntJWutBVI-jRHpQiqS3C/[IE224]_Final_Project


#3 Đọc dữ liệu

In [ ]:
data = pd.read_csv(DATA_PATH)

In [ ]:
data.head()

,building_type,squares,nums_bedroom,nums_wc,nums_floor,furniture,price,project_names,streets,wards,districts
0,Căn hộ chung cư,88.0,3,2,1,Cơ bản,9.5,Dự án The Western Capital,Đường Lý Chiêu Hoàng,Phường 10,6
1,Căn hộ chung cư,88.0,3,2,1,Đầy đủ,9.0,Dự án The Western Capital,Đường Lý Chiêu Hoàng,Phường 10,6
2,Căn hộ chung cư,70.0,2,2,1,Đầy đủ,7.0,Dự án The Western Capital,Đường Lý Chiêu Hoàng,Phường 10,6
3,Căn hộ chung cư,92.0,2,2,1,Đầy đủ,18.0,Dự án Vinhomes Central Park,Đường Điện Biên Phủ,Phường 22,Bình Thạnh
4,Căn hộ chung cư,56.0,3,2,1,Cao cấp,15.0,Dự án Vinhomes Central Park,Đường Điện Biên Phủ,Phường 22,Bình Thạnh


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1519 entries, 0 to 1518
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   building_type  1519 non-null   object 
 1   squares        1519 non-null   float64
 2   nums_bedroom   1519 non-null   int64  
 3   nums_wc        1519 non-null   int64  
 4   nums_floor     1519 non-null   int64  
 5   furniture      1519 non-null   object 
 6   price          1519 non-null   float64
 7   project_names  1519 non-null   object 
 8   streets        1519 non-null   object 
 9   wards          1519 non-null   object 
 10  districts      1519 non-null   object 
dtypes: float64(2), int64(3), object(6)
memory usage: 130.7+ KB


#4 Onehot-encoder các cột object



In [ ]:
object_cols = data.loc[:, data.dtypes == object].columns

data_temp = data.copy()
onehot_dict = {}

for col in object_cols:
  enc = OneHotEncoder()
  enc.fit(data_temp[col].values.reshape(-1,1))
  onehot_dict[col] = enc
  transformed = enc.transform(data_temp[col].values.reshape(-1,1)).toarray()
  ohe_df = pd.DataFrame(transformed, columns=enc.get_feature_names_out())
  data_temp = data_temp.drop([col],axis = 1)
  data_temp = data_temp.join(ohe_df)

In [ ]:
data_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1519 entries, 0 to 1518
Columns: 766 entries, squares to x0_Tân Phú
dtypes: float64(763), int64(3)
memory usage: 8.9 MB


#5 Tách dữ liệu thành tập train và test

In [ ]:
X = data_temp.drop('price',axis= 1)
y = data_temp['price']

In [ ]:
X

,squares,nums_bedroom,nums_wc,nums_floor,x0_Căn hộ chung cư,x0_Nhà riêng,x0_Nhà trọ phòng trọ,x0_Cao cấp,x0_Có tiện nghi cơ bản,x0_Cơ bản,...,x0_7,x0_8,x0_9,x0_Bình Thạnh,x0_Bình Tân,x0_Gò Vấp,x0_Phú Nhuận,x0_Thủ Đức,x0_Tân Bình,x0_Tân Phú
0,88.0,3,2,1,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,88.0,3,2,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,70.0,2,2,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,92.0,2,2,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,56.0,3,2,1,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1514,104.0,6,6,5,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1515,64.0,4,4,4,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1516,64.0,3,4,3,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1517,48.0,3,3,3,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

#6 Huấn luyện mô hình

In [ ]:
pipelines = {
    'linear': [('model', LinearRegression())],
    'xgboost': [('model',xgb.XGBRegressor(
    objective='reg:squarederror',  # for regression
    ))],
    'histogram': [('model',HistGradientBoostingRegressor())],
    'catboost': [('model',CatBoostRegressor())],
    'SVR_linear': [('model',SVR(kernel='linear'))],
    'SVR_poly': [('model',SVR(kernel='poly'))],
    'SVR_rbf': [('model',SVR(kernel='rbf'))],
    'lightGBM': [('model',LGBMRegressor())]
}

In [ ]:
results = []

for name, pipeline in pipelines.items():

  model = Pipeline(pipeline).fit(X_train, y_train)

  y_pred = model.predict(X_test)
  r2 = r2_score(y_test, y_pred)
  mse = mean_squared_error(y_test, y_pred)
  rmse = mean_squared_error(y_test, y_pred, squared=False)
  mae = mean_absolute_error(y_test, y_pred)


  results.append({
        'model': name,
        'r2': r2,
        'mse': mse,
        'rmse': rmse,
        'mae': mae,
        'model_train': model
    })

Learning rate set to 0.04134
0:	learn: 10.8386740	total: 2.89ms	remaining: 2.89s
1:	learn: 10.5842976	total: 5.09ms	remaining: 2.54s
2:	learn: 10.3164968	total: 7.7ms	remaining: 2.56s
3:	learn: 10.0702743	total: 10.1ms	remaining: 2.52s
4:	learn: 9.8164356	total: 12.5ms	remaining: 2.49s
5:	learn: 9.5894511	total: 14.9ms	remaining: 2.47s
6:	learn: 9.3790013	total: 17.3ms	remaining: 2.45s
7:	learn: 9.2006646	total: 19.6ms	remaining: 2.43s
8:	learn: 9.0082524	total: 22ms	remaining: 2.42s
9:	learn: 8.8252536	total: 24.4ms	remaining: 2.42s
10:	learn: 8.6558505	total: 26.8ms	remaining: 2.41s
11:	learn: 8.4920596	total: 29.2ms	remaining: 2.4s
12:	learn: 8.3461137	total: 31.5ms	remaining: 2.39s
13:	learn: 8.1998910	total: 33.8ms	remaining: 2.38s
14:	learn: 8.0466599	total: 36.2ms	remaining: 2.38s
15:	learn: 7.9192583	total: 38.5ms	remaining: 2.37s
16:	learn: 7.7916793	total: 40.8ms	remaining: 2.36s
17:	learn: 7.6662367	total: 43.3ms	remaining: 2.36s
18:	learn: 7.5537201	total: 45.6ms	remaining:

In [ ]:
pd.DataFrame(results).sort_values(by=['r2', 'mse'],
                                  ascending=[False, True]).reset_index(drop=True)

,model,r2,mse,rmse,mae,model_train
0,catboost,7.979083e-01,2.714002e+01,5.209609e+00,3.070878e+00,(<catboost.core.CatBoostRegressor object at 0x...
1,lightGBM,7.834234e-01,2.908527e+01,5.393076e+00,3.297596e+00,(LGBMRegressor())
2,histogram,7.802012e-01,2.951800e+01,5.433047e+00,3.291720e+00,(HistGradientBoostingRegressor())
3,xgboost,7.782595e-01,2.977877e+01,5.456993e+00,3.123120e+00,"(XGBRegressor(base_score=None, booster=None, c..."
4,SVR_linear,7.367221e-01,3.535706e+01,5.946180e+00,3.415363e+00,(SVR(kernel='linear'))
5,SVR_rbf,6.612607e-01,4.549119e+01,6.744716e+00,4.231545e+00,(SVR())
6,SVR_poly,3.106042e-01,9.258280e+01,9.621996e+00,5.097984e+00,(SVR(kernel='poly'))
7,linear,-7.850790e+17,1.054326e+20,1.026804e+10,1.578550e+09,(LinearRegression())
